In [123]:
import numpy as np
import matplotlib.pyplot as plt
import numexpr as ne
import scipy.signal as signal
import time
%matplotlib qt5

In [2]:
rampsPerFile = 1500
samplesPerRamp = 6000
samplesPerUpRamp = 3000
nbFiles = 10

fmin = 5.725e9
fMAX = 5.875e9
fc = 5.8e9
B = fMAX - fmin
T = 600e-6
alpha = (fMAX - fmin) / T
c = 3e8
kc = 4 * np.pi * fc / c

V = 40

print( "kc = {:.2f}, B = {}".format(kc, B) )

kc = 242.95, B = 150000000.0


In [3]:
Naz = nbFiles * rampsPerFile
xa_vec = np.arange( Naz ) * T * V

### Image geometry

In [68]:
x_min = 0
x_max = 300
r_min = 0
r_max = 200

#extent=[horizontal_min,horizontal_max,vertical_min,vertical_max]
extent=[r_min,r_max,x_max,x_min]

d_x = 1
d_r = 1

x = np.arange( x_min, x_max, d_x )
x = x.reshape( x.size, 1 )
r = np.arange( r_min, r_max, d_r )
r = r.reshape( 1, r.size )

### Read samples

In [5]:
data_dir = "/home/pleroy/DATA/SIMU"

In [149]:
firstFile = 0
lastFile = 9
adc = np.load(data_dir + '/adc_files_{}_{}.npy'.format(firstFile, lastFile) ) 
print( "adc.shape = {}".format( adc.shape ) )

adc.shape = (15000, 3000)


In [150]:
# conversion to analytic signal
az_corner = 150
ramp_corner = int( az_corner / (V * T) )
adc0 = adc[ramp_corner, ::2]
ifft_adc0 = np.fft.ifft( adc0 )
print("ramp_corner = {}".format(ramp_corner))

ramp_corner = 6250


In [151]:
plt.figure()

plt.subplot(221)
plt.plot(20 * np.log10( np.abs( ifft_adc0 ) ) )
plt.grid()

plt.subplot(222)
plt.plot(np.unwrap( np.angle( ifft_adc0 ) ) )
plt.grid()

plt.subplot(223)
plt.plot(np.real(ifft_adc0))
plt.grid()

plt.subplot(224)
plt.plot(np.imag(ifft_adc0))
plt.grid()

In [152]:
srf = np.fft.ifft( adc[:, ::2], axis=1 )
print( "srf.shape = {}".format(srf.shape))

srf.shape = (15000, 1500)


In [153]:
overSamplingRatio = 4
nbPointsResampled = overSamplingRatio * srf.shape[1]
rangeResolution = c / (2 * B)
r_oversampled = np.arange( 0, nbPointsResampled ) * rangeResolution / overSamplingRatio
print( "range from {:.2f}m to {:.2f}m, resolution = {}m, oversampled = {}m, ".format(
    r_oversampled[0], r_oversampled[-1], rangeResolution, rangeResolution / overSamplingRatio ) )

range from 0.00m to 1499.75m, resolution = 1.0m, oversampled = 0.25m, 


### Backprojection

In [154]:
nbLoops = xa_vec.size
loop = 0
img  = np.zeros( (x.size, r.size), dtype=complex )
print( "img.shape = {}, x.shape = {}, r.shape = {}".format( img.shape, x.shape, r.shape ) )

t = time.time()

for xa in xa_vec:
    if loop%1000 == 0:
        print( "{} / {}".format(loop, nbLoops ) )
    y = np.exp( 1j * kc * (r**2 + (xa-x)**2 )**0.5 ) \
    * np.interp( (r**2 + (xa-x)**2 )**0.5, r_oversampled, signal.resample( srf[loop,:], nbPointsResampled  ) )
    img += img_xa( xa )
    loop += 1
# np.exp( -1j * kc * r)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

img.shape = (300, 200), x.shape = (300, 1), r.shape = (1, 200)
0 / 15000
1000 / 15000
2000 / 15000
3000 / 15000
4000 / 15000
5000 / 15000
6000 / 15000
7000 / 15000
8000 / 15000
9000 / 15000
10000 / 15000
11000 / 15000
12000 / 15000
13000 / 15000
14000 / 15000
execution time = 75.93986654281616


In [155]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( img ) ), cmap='jet', extent=extent )
plt.grid()
plt.colorbar(orientation="horizontal")

In [55]:
plt.figure()
plt.plot( 20 * np.log10( np.abs( img[ 98, :] ) ) )
plt.plot( 20 * np.log10( np.abs( img[ 98, :] ) ) )
plt.plot( 20 * np.log10( np.abs( img[ 100, :] ) ) )
plt.plot( 20 * np.log10( np.abs( img[ 101, :] ) ) )
plt.grid()

### Resample

In [24]:
plt.imshow( 20 * np.log10( np.abs( srf[::10, :] ) ) )

In [ ]:
nbPointsResampled = 4 * srf.shape[1]
srf_resampled = signal.resample( srf[0,:], nbPointsResampled  )
print( "srf.shape = {}, srf_resamples.shape = {}".format( srf.shape, srf_resampled.shape ) )

In [ ]:
up_vec  = np.zeros( nbPointsResampled, dtype = complex )
vec_RD  = np.zeros( nbPointsResampled, dtype = complex  )

Nf = srf.shape[1]

vec_ind = int( np.ceil( ( Nf + 1 ) / 2 ) )

pos = 0
#upsample raw data
vec_RD[ 0 : vec_ind ] = adc[ pos, 0 : vec_ind ]
vec_RD[ - ( Nf - vec_ind ) : ] = adc[ pos, vec_ind : ]
# vec_RD contains RD(pos,:) and zeros: up-sampling

up_vec = np.fft.ifft( vec_RD ) # Transformation to time domain

print( "adc.shape = {}, vec_ind = {}, vec_RD.shape = {}".format( adc.shape, vec_ind, vec_RD.shape ) )

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot( np.arange( srf.shape[1] ), np.real( srf[0,:] ), 'go-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.real( srf_resampled ), 'b.-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.real( up_vec ) * 4, '.-', color="pink" )
ax = plt.gca()
plt.subplot(212, sharex=ax)
plt.plot( np.arange( srf.shape[1] ), np.imag( srf[0,:] ), 'go-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.imag( srf_resampled ), 'b.-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.imag( up_vec ) * 4, '.-', color="pink" )